In [ ]:
library(modules)
library(tidyverse)
library(lubridate)
library(caret)
library(ggplot2)
library(dplyr)
library(readxl)


In [ ]:
convert_mes <- function(mes) {
    return(switch(mes,
        "Enero" = 1,
        "Febrero" = 2,
        "Marzo" = 3,
        "Abril" = 4,
        "Mayo" = 5,
        "Junio" = 6,
        "Julio" = 7,
        "Agosto" = 8,
        "Septiembre" = 9,
        "Octubre" = 10,
        "Noviembre" = 11,
        "Diciembre" = 12
    ))
}

dataframe_ventas <- read_excel("../sources/act_dataset_ventas.xlsx", sheet = 1, col_names = FALSE)[, -1] %>%
    t() %>%
    as.data.frame()

colnames(dataframe_ventas) <- c("mes", "producto", "cantidad_ventas")
rownames(dataframe_ventas) <- NULL

dataframe_ventas <- dataframe_ventas %>%
    mutate(
        mes = as.integer(lapply(dataframe_ventas$mes, convert_mes)),
        producto = as.factor(producto),
        cantidad_ventas = as.double(cantidad_ventas)
    ) %>%
    complete(mes, producto, fill = list(cantidad_ventas = 0)) %>%
    mutate(
        trimestre = as.integer(trunc((mes - 1) / 3) + 1)
    )

head(dataframe_ventas)


dataframe_precios <- read_excel("../sources/act_dataset_ventas.xlsx", sheet = 2, col_names = FALSE)[, -1] %>%
    t() %>%
    as.data.frame()

colnames(dataframe_precios) <- c("producto", "costo", "precio_venta")
rownames(dataframe_precios) <- NULL

dataframe_precios <- dataframe_precios %>%
    mutate(
        producto = as.factor(producto),
        costo = as.double(costo),
        precio_venta = as.double(precio_venta),
        beneficio_unitario = precio_venta - costo
    )

head(dataframe_precios)

dataframe_output <- merge(dataframe_ventas, dataframe_precios, "producto") %>%
    arrange(mes, producto) %>%
    mutate(
        beneficio = beneficio_unitario * cantidad_ventas
    )

head(dataframe_output, n = 10000)

dataframe_output_trimestral <- dataframe_output %>%
    group_by(producto, trimestre) %>%
    summarise(beneficio = sum(beneficio))

head(dataframe_output_trimestral, n = 10000)
